# Orders

Gran parte de nuestra tarea es crear "Features", variables en los datos que representan un aspecto del fenómeno que estudiamos y nos ayudan a explicarlo.

**Ejercicio**

Vamos a investigar la relación entre orders y el score.

La idea es crear un dataframe con las siguientes features:

| feature_name 	                | type 	| description 	                                                                                                                                         |
|:------------------------------|:---:	|:------------------------------------------------------------------------------------------------------------------------------------------------------|
| `order_id` 	                  | str 	| the id of the order 	                                                                                                                                 |
| `tiempo_de_espera` 	          | float 	| numero de días entre order_date y delivered_date 	                                                                                                    |
| `tiempo_de_espera_previsto` 	 | float 	| numero de días entre order_date y estimated_delivery_date 	                                                                                           |
| `real_vas_esperado` 	         | float 	| si la fecha de entrega real es posterior a la fecha de entrega estimada, devuelve el número de días entre las dos fechas; de lo contrario, devuelve 0 |
| `order_status` 	              | str 	| statusde la orden	                                                                                                                                    |
| `es_cinco_estrellas` 	        | int 	| 1 si tuvo cinco estrellas en la  review, 0 caso contrario 	                                                                                           |
| `es_una_estrella` 	           | int 	| 1 si tuvo una estrella, 0 caso contrario 	                                                                                                            |
| `review_score` 	              | int 	| from 1 to 5 	                                                                                                                                         |
| `numero_de_productos` 	       | int 	| numero de productos que contiene una orden	                                                                                                           |
| `numero_de_vendedores` 	      | int 	| numero de vendedores unicos                                                                                                                           |
| `precio` 	                    | float 	| total del precio pagado por el cliente                                                                                                                |
| `valor_transporte` 	          | float 	| el valor del freight pagado por el cliente 	                                                                                                          |
| `distancia_al_cliente` 	      | float 	| distancia en km entre cliente y vendedor 	                                                                                                  |
  
Queremos filtrar las ordenes no entregadas

## Metodología:

Vamos a crear dos notebooks una de testeo y otra para importar nuestro paquete preprocess.orders_data y de ahí ir elavorando los métodos.

1er Método: Load_data: Trae el dataset sin nulos de una ubicación ya prefijada.

###  Buenas Pracitcas


Las notebooks se pueden hacer muy largas y desordenadas.
- Codifique su lógica para que siempre se pueda ejecutar de arriba a abajo sin fallar (Celda --> Ejecutar todo)
- Nombre sus variables con lo que significa y para que otra persona pueda entenderlo.
- Use nombres como `tmp` o `_` para pasos intermedios si sabe que no va a usarlos mucho tiempo
- Borre su código y combine celdas cuando sea relevante para minimizar el tamaño del Notebook (`Shift-M`)
- Oculte la salida de su celda si ya no necesita verla (haga doble clic en la sección roja `Out[]:` a la izquierda de su celda).
- Utilice la Structure de pycharm o Juypter lab para organizarse
- Shortcuts
    - `a` para insertar una celda arriba
    - `b` para insertar una celda debajo
    - `dd` para borrar una celda
    - `esc` y `flechas` para moverse entre celdas
    - `Shift-Enter` para ejecutar la celda y mover el foco a la siguiente
    - `Shift + Tab` cuando entre paréntesis de método, p. ¡`groupby()` para obtener los documentos! Repita varias veces para abrirlo permanentemente

In [134]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [136]:
# Import olist data
from olist.load_data import load_all_data
data = load_all_data("data/")

In [137]:
data.keys()

dict_keys(['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation', 'tablon_primario'])

## 1. Crear orders.py

### a) `tiempo_de_espera`
    Armar un dataframe que tenga
           order_id, tiempo_de_espera, tiempo_de_espera_previsto, real_vs_esperado, status_de_la_orden


# Pasos a seguir

> 1. Explora el dataframe `orders`
2. Filtra por `delivered orders`
3. Manipulá correctamente `datetime` (recordá el examen)
    - Repasá[`datetime`](https://docs.python.org/3/library/datetime.html)
    - Métodos para pasar "string" type a "pandas.datetime'  [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
4. Crea la columna `tiempo_de_espera`
5. Crea la columna  `tiempo_de_espera_previsto`
6. Crea la columna  `real_vs_esperado`
7. Chequea el nuevo dataset
8. Una vez terminado el código pasalo a tu librería `order.py`

Tanto para `tiempo_de_espera` como para `tiempo_de_espera_previsto`, debe restar las fechas/marcas de tiempo relevantes para obtener la diferencia horaria entre los objetos `pandas.datetime`. Luego, puede usar [`datetime.timedelta()`](https://docs.python.org/3/library/datetime.html#timedelta-objects) o [`np.timedelta64()`](https ://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-and-timedelta-arithmetic) para averiguar cuántos días representa esa resta.


In [138]:
orders = data['orders'].copy() # buena practica para estar seguro que no modificas tu dataset original

In [139]:
orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

Filtro solo las columnas que su status sea delivered

In [140]:
orders = orders[orders["order_status"] == "delivered"]

Paso todas las columnas que deberían ser fecha a su formato correspondiente

In [141]:
from olist.preprocess import transformar_columnas_datetime
orders = transformar_columnas_datetime(orders)

In [142]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

### Crear una columna
Es lo mismo utilizar .loc para asignar una columna nueva, que utilizar .assign o simplemente df['columna'] = operación

##  Compute just the number of days in each time_delta 
import datetime
one_day_delta = datetime.timedelta(days=1) # lleva el time delta a un día
one_day_delta = np.timedelta64(24, 'h') # también se puede hacer de esta manera


* \ se puede usar para ahorrar espacio
orders.loc[:,'tiempo_de_espera'] = \
    (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']) / one_day_delta

(orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.da


# pasarlo a modulo

Crear la funcion tiempo_de_espera e implementarla

to write python code


def tiempo_de_espera(orders, is_delivered=True):
    # filtrar por entregados y crea la varialbe tiempo de espera
    if is_delivered:
        orders = orders.query("order_status=='delivered'").copy()
    # compute wait time
    orders.loc[:, 'tiempo_de_espera'] = \
        (orders['order_delivered_customer_date'] -
         orders['order_purchase_timestamp']) / np.timedelta64(24, 'h')
    return orders

def

Filtro solo aquellas columnas que voy a necesitar

In [143]:
orders = orders[['order_id', 'order_status', 'order_purchase_timestamp',
                'order_delivered_customer_date', 'order_estimated_delivery_date']]

Creo las columnas tiempo_de_espera y tiempo_de_espera_previsto como diferencia entre fechas, asignando también el tipo de dato que se desea

In [144]:
from olist.preprocess import tiempo_de_espera
orders = tiempo_de_espera(orders)
orders.head()

,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,tiempo_de_espera,tiempo_de_espera_previsto
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,8.0,15.0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,13.0,19.0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,9.0,26.0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,13.0,26.0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,2.0,12.0


Creo la columna real_vs_esperado y despues le asigno un 0 a todas las columnas que no sean numeros positivos

In [145]:
from olist.preprocess import real_vs_esperado
orders = real_vs_esperado(orders)
orders

,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,8.0,15.0,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,13.0,19.0,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,9.0,26.0,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,13.0,26.0,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,2.0,12.0,0.0
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,delivered,2017-03-09 09:54:05,2017-03-17 15:08:01,2017-03-28,8.0,18.0,0.0
99437,63943bddc261676b46f01ca7ac2f7bd8,delivered,2018-02-06 12:58:58,2018-02-28 17:37:56,2018-03-02,22.0,23.0,0.0
99438,83c1379a015df1e13d02aae0204711ab,delivered,2017-08-27 14:46:43,2017-09-21 11:24:17,2017-09-27,24.0,30.0,0.0
99439,11c177c8e97725db2631073c19f07b62,delivered,2018-01-08 21:28:27,2018-01-25 23:32:54,2018-02-15,17.0,37.0,0.0


Elimino columnas innecesarias

In [146]:
orders.drop('order_purchase_timestamp', axis=1, inplace=True)
orders.drop('order_delivered_customer_date', axis=1, inplace=True)
orders.drop('order_estimated_delivery_date', axis=1, inplace=True)

Renombro para que quede con el formato adecuado

In [147]:
orders.rename(columns={'order_status': 'status_de_la_orden'}, inplace=True)

In [148]:
orders.dtypes

order_id                      object
status_de_la_orden            object
tiempo_de_espera             float64
tiempo_de_espera_previsto    float64
real_vs_esperado             float64
dtype: object

In [149]:
orders

,order_id,status_de_la_orden,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,8.0,15.0,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,13.0,19.0,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,9.0,26.0,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,13.0,26.0,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2.0,12.0,0.0
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,delivered,8.0,18.0,0.0
99437,63943bddc261676b46f01ca7ac2f7bd8,delivered,22.0,23.0,0.0
99438,83c1379a015df1e13d02aae0204711ab,delivered,24.0,30.0,0.0
99439,11c177c8e97725db2631073c19f07b62,delivered,17.0,37.0,0.0


Chequeo que me deje importar el dataset

In [150]:
from olist.order import orders

In [151]:
orders

,order_id,status_de_la_orden,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,8.0,15.0,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,13.0,19.0,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,9.0,26.0,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,13.0,26.0,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2.0,12.0,0.0
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,delivered,8.0,18.0,0.0
99437,63943bddc261676b46f01ca7ac2f7bd8,delivered,22.0,23.0,0.0
99438,83c1379a015df1e13d02aae0204711ab,delivered,24.0,30.0,0.0
99439,11c177c8e97725db2631073c19f07b62,delivered,17.0,37.0,0.0


### b) `puntaje_de_compra`
     retorna un dataframe con
        order_id, es_cinco_estrellas, es_una_estrella, review_score

recordar del examen `DataFrame.apply()` o tmb se puede usar`Series.map()`


👉 We load the `reviews` for you

In [152]:
reviews = data['order_reviews'].copy()

reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [153]:
reviews.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

Me quedo solo con las columnas que voy a necesitar

In [154]:
reviews = reviews[["order_id", "review_score"]]

Creo las nuevas columnas

In [155]:
from olist.preprocess import puntaje_de_compra
reviews = puntaje_de_compra(reviews)

Chequeo que funcionen bien mediante un sample

In [156]:
reviews.sample(4)

,order_id,review_score,es_cinco_estrellas,es_una_estrella
23984,098f60afb74eab906fedaf802a833a97,4,0,0
21633,508f1058f7c9037c12fa29baf78dc417,5,1,0
90678,8b249044e8ce6377509b32dc840bfe76,5,1,0
41703,79b8758142105381b27a6b990f80bf3d,5,1,0


### c) `numero_de_productos`:
     retorna un dataframe con
        order_id, numero_de_productos (numero_de_productos_por_orden)

In [157]:
order_items = data['order_items'].copy()

In [158]:
order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


Agrupo y cuento cuantas veces se repite el order_id para saber la cantidad de productos que contiene

In [159]:
from olist.preprocess import calcular_numero_productos
#fijarse que no uso solo un dataframe
numero_de_productos = calcular_numero_productos(order_items)
numero_de_productos

,order_id,numero_de_productos
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


In [160]:
numero_de_productos.dtypes

order_id               object
numero_de_productos     int64
dtype: object

### d) `numero_de_vendedores`:
     Retorna un dataframe con
        order_id, numero_de_vendedores (total de vendedores UNICOS !!! )

HINT!:
`pd.Series.nunique()`


In [161]:
data.keys()

dict_keys(['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation', 'tablon_primario'])

In [162]:
order_items = data['order_items'].copy()

In [163]:
order_items.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

Agrupo por order_id y cuento los valores unicos de vendedores por cada orden

Renombro la columna para que el título sea más representativo

In [164]:
from olist.preprocess import numero_de_vendedores
numero_de_vendedores = numero_de_vendedores(order_items)
numero_de_vendedores

,order_id,numero_de_vendedores
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


### e) `calcular_precio_y_transporte`
     Returns a DataFrame with:
        order_id, precio, valor_transporte

recordar el agg en el groupby

In [165]:
data.keys()

dict_keys(['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation', 'tablon_primario'])

In [166]:
order_items = data['order_items'].copy()

Sumo cada línea de la orden para obtener el precio total de la orden y el precio total del transporte de la orden

In [167]:
from olist.preprocess import calcular_precio_y_transporte
precio_y_transporte = calcular_precio_y_transporte(order_items)
precio_y_transporte

,order_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,299.99,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,350.00,36.53
98663,fffce4705a9662cd70adb13d4a31832d,99.90,16.95
98664,fffe18544ffabc95dfada21779c9644f,55.99,8.72


### e) `distancia_al_cliente`

    Retorna un dataframe con
        order_id, distancia_al_cliente (distancia en kilometros)

usar`haversine_distance`

Veo que columnas tiene cada csv para saber cuales voy a necesitar

In [168]:
for nombre in data.keys():
    print(nombre)
    nombre = data[nombre].copy()
    print(nombre.columns)

customers
Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
geolocation
Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state'],
      dtype='object')
orders
Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
order_items
Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
order_payments
Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')
order_reviews
Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'revie

In [169]:
from olist.preprocess import haversine_distance

In [170]:
sellers = data['sellers'].copy()
geolocation = data['geolocation'].copy()
customers = data['customers'].copy()
order_items = data['order_items'].copy()
orders = data['orders'].copy()

In [171]:
from olist.preprocess import calcular_distancia_vendedor_comprador
distancias = calcular_distancia_vendedor_comprador(orders, geolocation, customers, order_items, sellers)
distancias

,order_id,distancia_al_cliente
0,00010242fe8c5a6d1ba2dd792cb16214,301.005664
1,00018f77f2f0320c557190d7a144bdd3,589.274140
2,000229ec398224ef6ca0657da4fc703e,312.495046
3,00024acbcdf0a6daa1e931b038114c75,301.951753
4,00042b26cf59d7ce69dfabb4e55b4fd9,646.221788
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,2755.314485
98662,fffcd46ef2263f404302a634eb57f7eb,354.848852
98663,fffce4705a9662cd70adb13d4a31832d,338.827218
98664,fffe18544ffabc95dfada21779c9644f,72.870476


cual es la mediana de la distancia

In [172]:
distancias['distancia_al_cliente'].describe()

count    98177.000000
mean       601.961534
std        596.877838
min          0.000000
25%        186.380313
50%        433.698770
75%        798.435455
max       8736.947618
Name: distancia_al_cliente, dtype: float64

# Merger todos los datasets excepto el de distancia

In [173]:
from olist.order import orders, reviews, numero_de_productos, numero_de_vendedores, precio_y_transporte, distancias

In [174]:
orders

,order_id,status_de_la_orden,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,8.0,15.0,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,13.0,19.0,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,9.0,26.0,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,13.0,26.0,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2.0,12.0,0.0
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,delivered,8.0,18.0,0.0
99437,63943bddc261676b46f01ca7ac2f7bd8,delivered,22.0,23.0,0.0
99438,83c1379a015df1e13d02aae0204711ab,delivered,24.0,30.0,0.0
99439,11c177c8e97725db2631073c19f07b62,delivered,17.0,37.0,0.0


In [175]:
tablon_primario = orders.merge(reviews, on='order_id') \
    .merge(numero_de_productos, on='order_id') \
    .merge(numero_de_vendedores, on='order_id') \
    .merge(precio_y_transporte, on='order_id')\
     .merge(distancias, on='order_id')
tablon_primario

,order_id,status_de_la_orden,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado,review_score,es_cinco_estrellas,es_una_estrella,numero_de_productos,numero_de_vendedores,price,freight_value,distancia_al_cliente
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,8.0,15.0,0.0,4,0,0,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,13.0,19.0,0.0,4,0,0,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,9.0,26.0,0.0,5,1,0,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,13.0,26.0,0.0,5,1,0,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2.0,12.0,0.0,5,1,0,1,1,19.90,8.72,30.174037
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95875,9c5dedf39a927c1b2549525ed64a053c,delivered,8.0,18.0,0.0,5,1,0,1,1,72.00,13.08,69.481037
95876,63943bddc261676b46f01ca7ac2f7bd8,delivered,22.0,23.0,0.0,4,0,0,1,1,174.90,20.10,474.098245
95877,83c1379a015df1e13d02aae0204711ab,delivered,24.0,30.0,0.0,5,1,0,1,1,205.99,65.02,968.051192
95878,11c177c8e97725db2631073c19f07b62,delivered,17.0,37.0,0.0,2,0,0,2,1,359.98,81.18,370.146853


In [176]:
tablon_primario.to_csv("data/tablon_primario.csv")